In [1]:
#!pip install -r requirements.txt
import numpy as np
from utils import *
from model import *
import rasterio as rio
import os
import matplotlib.pyplot as plt
import subprocess
from PIL import Image
from tqdm import tqdm
from utils import *
%matplotlib inline
%load_ext autoreload


The autoreload module is not an IPython extension.


##### Observe the full images
We load train and test data, rebuild the full images using the merge_raster.py file and save them

In [2]:
file_path = "glaciers_mapping_downsampled"
data_paths = {}

#build paths for each pipeline
for pipeline in ["train", "test"]:
    for date in [0,1]:
        data_paths[f"{pipeline}{date}"] = f"{file_path}/{pipeline}/date{date}"

In [3]:
#reconstruct full images
for key, path in data_paths.items():
    output_path = f"{file_path}/{key}_merged.tif"
    # Execute the command to merge rasters using subprocess and save them
    command = ["python", "glaciers_mapping_downsampled/merge_rasters.py", "-i", path, "-o", output_path]
    subprocess.run(command)

Then we save the RGB images in png to be able to observe them and (maybe) dras useful conclusions

In [4]:
for key in data_paths.keys():
    img_path = f"{file_path}/{key}_merged"
    img = rio.open(f"{img_path}.tif").read()
    normalized_image = (img - np.min(img)) / (
                            np.max(img) - np.min(img)) # maybe not the best way to normalize?
    rgb_image = (normalized_image[:3] * 255).astype(np.uint8).transpose(1, 2, 0)
    pil_image = Image.fromarray(rgb_image)
    save_path = f"full_rgb_images/{key}.png"
    pil_image.save(save_path)

### Loading all patches

We store the patches in dictionaries : {( (position_x_of_patch, position_y_of_patch) : numpy_array_of_the_patch )}

At this stage the np array of patches are of dimension (band, pixel_x, pixel_y) i.e. (4,128,128)

In [5]:
patches_train0 = get_organized_dict_of_patches(f"{data_paths['train0']}")
patches_train1 = get_organized_dict_of_patches(f"{data_paths['train1']}")
patches_test0 = get_organized_dict_of_patches(f"{data_paths['test0']}")
patches_test1 = get_organized_dict_of_patches(f"{data_paths['test1']}")

patches_train_gt = get_organized_dict_of_patches("glaciers_mapping_downsampled/train/gt")
patches_test_gt = get_organized_dict_of_patches("glaciers_mapping_downsampled/test/gt")

100%|██████████| 351/351 [00:00<00:00, 441.72it/s]


### Preprocessing

In [6]:
print(patches_test0[(1,3)].shape) #shape of test patch 1_3 
print(patches_test_gt[(1,3)].shape) #shape of test ground ruth patch 1_3

(4, 128, 128)
(1, 128, 128)


### Data Augmentation